In [ ]:
try:
    import pandas as pd
    from pyedgar.utilities import edgarweb
except ModuleNotFoundError:
    print("Warning, pyedgar not found.")
    class _o_(object):
        def edgar_links(*args, **kwargs):
            return ''
    edgarweb = _o_()

try:
    import panda_cub
    panda_cub.monkey_patch_pandas()
    panda_cub.monkey_patch_seaborn()
except ModuleNotFoundError:
    print("Warning, panda_cub not found.")

In [ ]:
print("linkhead(df, n=5, fields=None, cik='cik', accession='accession')")
def linkhead(df, n=5, fields=None, cik='cik', accession='accession', return_df=False):
    """
    Displays top rows of a dataframe, and includes
    links to the HTML and FTP websites if CIK and Accession are found.
    """
    w = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', -1)
    
    if fields is None:
        fields = list(df.columns)
    
    dfn = df.head(n).copy()        
    
    if cik in dfn.columns:
        linkstr, i = 'links', 0
        while linkstr in dfn.columns:
            linkstr = 'links%d' % i
            i += 1
        dfn[linkstr] = dfn.apply(lambda row: edgarweb.edgar_links(row[cik], row[accession]), axis=1)
        fields.append(linkstr)
    
    html = dfn[fields].to_html(escape=False, index=False, na_rep="")
    
    display_html(html, raw=True)
    pd.set_option('display.max_colwidth', w)
    
    if return_df: 
        return dfn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

print("timehist(dtseries_or_df, time_variable='year', y_tic_number=4, x_tic_skip=0, *args, **kwargs)")
def timehist(dtseries_or_df, time_variable='year',
             y_tic_number=4, x_tic_skip=0,
             width=.9, ax=None, skip_retick=False,
             *args, **kwargs):
    """
    Historgam of observations per time period.
    First tries: dtseries_or_df.dt.time_variable
    Failing that, does dtseries_or_df.value_counts()
    Sends args and kwargs to figure.
    """
    x_tic_skip += 1
    sns.set_style('darkgrid')
    sns.set_context('talk', rc={'patch.linewidth': 0, 'patch.edgecolor': 'k', 'patch.facecolor': 'k'})
    _d = dtseries_or_df
    try:
        _d = _d.dt.__getattribute__(time_variable)
    except:
        try:
            _d = _d[time_variable]
        except:
            pass
    _g = _d.value_counts().sort_index()
    if len(_g) > 1000:
        logger.error("ERROR: You are trying to plot something with too many levels. Don't do that.")
        return 
    
    if ax is None:
        if 'figsize' not in kwargs:
            kwargs['figsize'] = (13,2)
        plt.figure(*args, **kwargs)
        ax = plt.gca()
    
    ax.bar(_g.index, _g, width=width)
    
    if not skip_retick:
        # Format and label X axis
        ax.set_xlim(left=_g.index.min()-0.5, right=_g.index.max()+0.5)
        _t = _g.index[::x_tic_skip]
        ax.set_xticks(_t)
        ax.set_xticklabels(map(str, _t), rotation='vertical')

        # Label Y Axis
        tene = math.log10(_g.max())//1-1
        topnum = math.ceil(_g.max() / 10**tene)
        ax.set_yticks([(topnum * i // y_tic_number)*10**tene for i in range(y_tic_number, 0, -1)])
    
    return ax